In [7]:
import numpy as np
import math
import tensorflow as tf


In [11]:
x=str(-1)
n=x[1:]
y=int(n)
print(y)


1


In [13]:
x=tf.constant([[0.0034,-0.12, 0.045, 0.2, 1, -1.05, 2.34, -0.44, 0.5],[0.0034,-0.12, 0.045, 0.2, 1, -1.05, 2.34, -0.44, 0.5]])
y=tf.reduce_max(x)
c=x/y
c

<tf.Tensor: shape=(2, 9), dtype=float32, numpy=
array([[ 0.00145299, -0.05128205,  0.01923077,  0.08547009,  0.42735043,
        -0.44871795,  1.        , -0.18803419,  0.21367522],
       [ 0.00145299, -0.05128205,  0.01923077,  0.08547009,  0.42735043,
        -0.44871795,  1.        , -0.18803419,  0.21367522]],
      dtype=float32)>

In [2]:
1/225


0.0044444444444444444

In [1]:
def LQ(tensor, bitwidth):
    # Create a mask for weights equal to 0
    #rounded_tensor=tf.where(tensor>0,tf.my u4xdcvp]
    # [6ath.floor(tensor),tf.math.ceil(tensor))
    rounded_weights=tensor*100
    rounded_weights=tf.where(rounded_weights>0,tf.math.floor(rounded_weights),tf.math.ceil(rounded_weights))
    epsilon = 1e-7

    # Compute weight_log using log2
    weight_log = tf.math.log(tf.math.abs(tensor)+epsilon) / tf.math.log(2.0)
    print(weight_log)
    weight_log=tf.where(weight_log<0,tf.math.ceil(weight_log),tf.math.floor(weight_log))
    bit_pot = -2 ** bitwidth
    mask_condition = weight_log >= bit_pot
    
    # Initialize the result tensor with zeros
    result = tf.zeros_like(tensor, dtype=tf.float32)
    
    # Apply the condition where weight_log is valid
    result = tf.where(rounded_weights!=0,tf.where(mask_condition, tf.where(tensor > 0, tf.math.abs(weight_log), weight_log), result),result)
    
    return result



In [14]:
lq=LQ(c,4)
lq


tf.Tensor(
[[-9.4266586e+00 -4.2853994e+00 -5.7004318e+00 -3.5484347e+00
  -1.2265083e+00 -1.1561189e+00  1.7198263e-07 -2.4109323e+00
  -2.2265079e+00]
 [-9.4266586e+00 -4.2853994e+00 -5.7004318e+00 -3.5484347e+00
  -1.2265083e+00 -1.1561189e+00  1.7198263e-07 -2.4109323e+00
  -2.2265079e+00]], shape=(2, 9), dtype=float32)


<tf.Tensor: shape=(2, 9), dtype=float32, numpy=
array([[ 0., -4.,  5.,  3.,  1., -1.,  0., -2.,  2.],
       [ 0., -4.,  5.,  3.,  1., -1.,  0., -2.,  2.]], dtype=float32)>

In [9]:
tf.shape(c)[0]

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [4]:
def bitshift(weight,shift):
   res=0
   if shift<0:
       y=np.abs(shift)
       res= weight>>shift
   else:
        res= weight<<shift
   def grad(dy):
       dw=2**shift*dy
       ds=2**shift*weight*tf.math.log(2)*dy
       return dw,ds
   return res,grad

In [2]:
def bitshift2(weight,shift):
    if shift<0:
       y=tf.math.abs(shift)
       return tf.bitwise.right_shift(weight,shift)
    else:
       return tf.bitwise.left_shift(weight,shift)


In [3]:
def MatrixBitShift(A,B):
    A=tf.cast(A,tf.int32)
    B=tf.cast(B,tf.int32)
    columns2=tf.shape(B)[1]
    rows2=tf.shape(B)[0]
    rows1=tf.shape(A)[0]
    C=tf.zeros(shape=(rows1,columns2),dtype=tf.int32)
    for i in range(0,rows1):
      for j in range(0,rows2):
            for  k in range(0,columns2):
                original=C[i][k]
                a=A[i][j]
                b=B[j][k]
                x=bitshift2(a,b)
                #print(x)
                update=original+x
                
                index=tf.constant([[i,k]])
                #u=tf.constant([update],dtype=tf.int32)
                C=tf.tensor_scatter_nd_update(C,index,[update])
    
    return C
    

In [6]:
tensor = tf.constant([[1, 1, 2, 3],[1,1,2,3]])
tens2=tf.constant([[1,1,1],[1,1,1],[1,1,1],[1,1,1]])
C=MatrixBitShift(tensor,tens2)
print(C)
#print(tensor[0][1])

tf.Tensor(
[[14 14 14]
 [14 14 14]], shape=(2, 3), dtype=int32)


In [11]:
def fixed_to_float(fixed_point_value, fractional_bits):
    """
    Convert a fixed-point number to floating-point.
    
    Args:
    fixed_point_value (int): The fixed-point value (signed or unsigned).
    fractional_bits (int): The number of fractional bits.
    
    Returns:
    float: The floating-point equivalent.
    """
    # Divide by 2^fractional_bits to convert to floating point
    float_value = fixed_point_value / (2 ** fractional_bits)
    return float_value


In [12]:
fixed_to_float(-1216,8)

-4.75

In [4]:
import os 
os.chdir("Air Dataset")

In [6]:
x=np.ones(1024, dtype=int)
with open('dense_biases.txt', 'w') as cf:
       np.savetxt(cf,x,delimiter=" ",fmt="%d")


In [15]:
def fixed_point_to_binary(value, integer_bits=8, fractional_bits=8):
    # Check if the input is within the range for a signed 16-bit fixed-point number
    max_value = (1 << (integer_bits + fractional_bits - 1)) - 1  # Maximum positive value
    min_value = -(1 << (integer_bits + fractional_bits - 1))     # Maximum negative value
    
    if not min_value <= value <= max_value:
        raise ValueError(f"Value must be between {min_value} and {max_value}")

    # Multiply by 2^fractional_bits to convert fractional part to an integer
    scaled_value = round(value * (1 << fractional_bits))

    # If the value is negative, convert it to two's complement representation
    if scaled_value < 0:
        scaled_value = (1 << (integer_bits + fractional_bits)) + scaled_value

    # Convert to binary and format as a 16-bit binary number
    binary_rep = format(scaled_value, f'0{integer_bits + fractional_bits}b')

    return binary_rep

# Example usage:
print(fixed_point_to_binary(4.75))   # Output: '0000010011000000' (binary representation of 4.75 in Q8.8)
print(fixed_point_to_binary())  # Output: '1111101101000000' (two's complement binary of -4.75 in Q8.8)


0000010011000000
-111100000000000000


In [8]:
def float32_to_fixed16(value, fractional_bits=8):
    # Scale by 2^fractional_bits
    scaled_value = value * (2 ** fractional_bits)
    
    # Round to the nearest integer
    fixed_value = np.round(scaled_value).astype(np.int16)
    
    # Handle overflow (for 16-bit signed integers)
    #max_val = 2**15 - 1
    #min_val = -2**15
    #fixed_value = np.clip(fixed_value, min_val, max_val)
    
    return fixed_value


In [9]:
float32_to_fixed16(-4.75,8)

-1216

In [2]:
def float_to_custom_32bit(value):
    # Separate the integer and fractional parts
    integer_part = int(value)
    fractional_part = value - integer_part

    # Convert integer part to binary (13 bits)
    if integer_part >= 2**13 or integer_part < -(2**13):
        raise ValueError("Integer part is out of 13-bit range")
    int_bin = format(integer_part & ((1 << 13) - 1), '013b')

    # Convert fractional part to binary (19 bits)
    frac_bin = ''
    fraction = fractional_part
    for _ in range(19):
        fraction *= 2
        bit = int(fraction)
        frac_bin += str(bit)
        fraction -= bit

    # Concatenate the integer and fractional parts
    custom_32bit_binary = int_bin + frac_bin

    return custom_32bit_binary

# Test the function
print(float_to_custom_32bit(6.5625))     # Output: '0000000000101001100000000000000'
print(float_to_custom_32bit(-3.125))   # Output: '1111111111101000000000000000000'



00000000001101001000000000000000
111111111110100-10000000000000000


In [1]:
def binary_to_twos_complement(binary_str):
    # Check if the input is a valid binary string
    if not all(bit in '01' for bit in binary_str):
        raise ValueError("Input must be a binary string.")
    
    # Find the number of bits
    num_bits = len(binary_str)
    
    # Convert the binary string to an integer
    value = int(binary_str, 2)
    
    # If the value is already negative (starts with '1'), just return the binary string
    if binary_str[0] == '1':
        return binary_str
    
    # Calculate the two's complement of the value by subtracting it from 2^num_bits
    twos_complement_value = (1 << num_bits) - value
    
    # Convert the two's complement value back to binary, format with leading zeros, and return
    return format(twos_complement_value, f'0{num_bits}b')

# Example usage
print(binary_to_twos_complement("00000111"))  # Output: '11111001' (two's complement of 7)
print(binary_to_twos_complement("00000001"))  # Output: '11111111' (two's complement of 1)


11111001
11111111


In [2]:
binary_to_twos_complement('0000010011000000')

'1111101101000000'

In [17]:
def int_to_twos_complement(value, num_bits):
    # Check if the value fits within the specified number of bits
    if not -(1 << (num_bits - 1)) <= value < (1 << (num_bits - 1)):
        raise ValueError(f"Value out of range for {num_bits}-bit representation.")
    
    # If the value is negative, convert to two's complement by adding 2^num_bits
    if value < 0:
        value = (1 << num_bits) + value
    
    # Convert the value to binary and format to the specified number of bits
    binary_rep = format(value, f'0{num_bits}b')
    
    return binary_rep

# Example usage:
print(int_to_twos_complement(7, 8))    # Output: '00000111' (7 in 8 bits)
print(int_to_twos_complement(-7, 8))   # Output: '11111001' (-7 in 8 bits)
print(int_to_twos_complement(-1216, 16))  # Output: '1111111111111001' (-7 in 16 bits)


00000111
11111001
1111101101000000
